# Analysis of A/B testing

Companies often run tens, if not hundreds, of A/B tests at the same time. 
In this project, I analyze results from an A/B test and design an algorithm to automate some steps.

For an e-commerce company with localized versions of the site, Spain-based users have a much higher conversion rate than any other Spanish-speaking country. One possible one reason could be translation. All Spanish- speaking countries had the same translation of the site which was written by a Spaniard. 

An A/B test to compare conversion rate of having the same translation for all countries, with that of having local translation for each country. That is, Argentinian users would see a translation written by an Argentinian, Mexican users by a Mexican and so on. Obviously, nothing would change for users from Spain.
The suprising results show that the test is negative. I.e., it appears that the non-localized translation was doing better.

### The goal of this project is to
* Confirm that the test is actually negative. That is, it appears that the old version of the site with just one translation across Spain and LatAm performs better
* Explain why that might be happening. Are the localized translations really worse?
* If you identified what was wrong, design an algorithm that would return FALSE if the same problem is happening in the future and TRUE if everything is good and the results can be trusted.

## Data

###  "test_table" - general information about the test results

Columns:
* user_id : the id of the user. Unique by user. Can be joined to user id in the other table. For each user, we just check whether conversion happens the first time they land on the site since the test started.
* date : when they came to the site for the first time since the test started
* source : marketing channel: Ads, SEO, Direct . Direct means everything except for ads and SEO. Such as directly typing site URL on the browser, downloading the app w/o coming from SEO or Ads, referral friend, etc.
* device : device used by the user. It can be mobile or web
* browser_language : in browser or app settings, the language chosen by the user. It can be EN, ES, Other (Other means any language except for English and Spanish) ads_channel : if marketing channel is ads, this is the site where the ad was displayed. It can be: Google, Facebook, Bing, Yahoo ,Other. If the user didn't come via an ad, this field is NA
* browser : user browser. It can be: IE, Chrome, Android_App, FireFox, Iphone_App, Safari, Opera
* conversion : whether the user converted (1) or not (0). This is our label. A test is considered successful if it increases the proportion of users who convert.
* test : users are randomly split into test (1) and control (0). Test users see the new translation and control the old one. For Spain-based users, this is obviously always 0 since there is no change there.

###  "user_table" - some information about the user

Columns:
* user_id : the id of the user. It can be joined to user id in the other table sex : user sex: Male or Female
* age : user age (self-reported)
* country : user country based on ip address

## 1. Exploratory Data Analysis and Feature Generation 
### 1.1 Read data from csv file

In [1]:
# import necessary pacakges
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn import base
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [8]:
# read data as a pandas dataframe
test = pd.read_csv("Translation_Test/test_table.csv")
print(test.shape)    # check size of the data, there are 31.6K user records
test.head()          # look at the first 5 rows of data

(453321, 9)


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [7]:
user = pd.read_csv("Translation_Test/user_table.csv")
print(user.shape)    # check size of the data, there are 31.6K user records
user.head()          # look at the first 5 rows of data

(452867, 4)


,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain
